In [ ]:
# clean last run result

!('./clean.sh')

In [ ]:
from thunder_ase.fireball import Fireball
import numpy as np
import ase
from ase.build import molecule
import timeit

In [ ]:
import sys
from ase import units

# Socket Communication
from ase.calculators.socketio import SocketIOCalculator

# ase BFGS optimization
from ase.optimize import BFGS

# ase MD modules
from ase.md.verlet import VelocityVerlet as NVE
from ase.md.nvtberendsen import NVTBerendsen

## Without Socket

In [ ]:
def run_without_socket():
    # Construct Structure
    atoms = molecule('CH3CH2OH')

    # set Fdata dir
    Fdata_path = '../../../Fdata/Fdata-Horsfield-0.10-9SN.Hs4.10-9DN.Cs4.35p4.80.Ns3.95p4.40.Os3.35p3.80'

    max_step = 10
    kwargs = {
        'scf_tolerance_set':1E-8,
    }
    fireball = Fireball(command='fireball.9.x', Fdata_path=Fdata_path, **kwargs)
    atoms.calc = fireball
    dyn = BFGS(atoms, trajectory='opt.traj', logfile='opt.log')
    dyn.run(fmax=0.01, steps=max_step)  # run until maximum force is 0.01 eV/A

In [ ]:
timeit.timeit(run_without_socket,number=1)

## With Socket

With socket will save lots of time on reading Fdata.

In [ ]:
!('./clean.sh')

In [ ]:
def run_with_socket():
    # Construct Structure
    atoms = molecule('CH3CH2OH')

    # set Fdata dir
    Fdata_path = '../../../Fdata/Fdata-Horsfield-0.10-9SN.Hs4.10-9DN.Cs4.35p4.80.Ns3.95p4.40.Os3.35p3.80'

    # Sockets
    unixsocket = 'thunder-ase'

    max_step = 10
    kwargs = {
              'ipi': 1,
              'nstepf':max_step+1,  # max step
              'inet': 0,
              'host': unixsocket,
              'scf_tolerance_set':1E-8,
              }
    fireball = Fireball(command='fireball-ase.9.x', Fdata_path=Fdata_path, **kwargs)
    fireball.prefix = ''
    dyn = BFGS(atoms, trajectory='opt_socket.traj', logfile='opt_socket.log')

    with SocketIOCalculator(fireball, log=None, unixsocket=unixsocket) as calc:
        atoms.calc = calc
        dyn.run(fmax=0.01, steps=max_step)  # run until maximum force is 0.01 eV/A

In [ ]:
timeit.timeit(run_with_socket,number=1)